In [2]:
from workflow.scripts.utils import load_CMIP_data, copy_meta_data_CMIP,transelate_aerocom_helper, regrid_global
from pyclim_noresm.general_util_funcs import yearly_avg
import time
import pathlib as pl
import xarray as xr

In [30]:
paths = snakemake.input

try:
    ds_am = load_CMIP_data(paths.air_mass, data_vars=['airmass'])
    ds_mmr = load_CMIP_data(paths.mmr, data_vars=['mmrdust'])
except TypeError:
    ds_am = load_CMIP_data(paths.air_mass, data_vars=['airmass'], use_cftime=True)
    ds_mmr = load_CMIP_data(paths.mmr, data_vars=['mmrdust'], use_cftime=True)

attrs = copy_meta_data_CMIP(ds_mmr.attrs)
wildcards = snakemake.wildcards
freq = transelate_aerocom_helper(wildcards)


params = snakemake.params
accumlative_vars = params.get('accumalative_vars',None)
variable=['loaddust']

In [48]:
ds_load = ds_mmr[ds_mmr.variable_id] * ds_am[ds_am.variable_id]
ds_load = ds_load.sum(dim='lev')
ds_load.attrs['units'] = ds_am[ds_am.variable_id].units
ds_load.attrs['long_name'] = 'Load of Dust'
ds_load = ds_load.to_dataset(name='loaddust')


In [49]:
def resample_data(ds):
    with xr.set_options(keep_attrs=True):
        if not ds.cf.bounds.get('lon', None):
            ds = ds.cf.add_bounds(['lon', 'lat'])
        if freq == 'Ayear':
            data=ds
            if 'time_bnds' in ds.data_vars:
                data = ds.drop_vars('time_bnds')
                #data[data.variable_id] = yearly_avg(data[data.variable_id])
                data=ds.resample(time='Y').mean()
                data.attrs['history'] = data.attrs['history'] + f', annual average'
                dvar_attrs = copy_meta_data_CMIP(data[wildcards.variable].attrs)
            data = data.assign({ds.cf.bounds['lon'][0]:ds[ds.cf.bounds['lon'][0]]})
            data = data.assign({ds.cf.bounds['lat'][0]:ds[ds.cf.bounds['lat'][0]]})
        elif freq == 'clim':
            t0 = data.time[0].dt.strftime('%Y/%m').values
            t1 = data.time[0].dt.strftime('%Y/%m').values
            data = data.groupby('time.month').mean('time')
            data[data.variable_id].attrs['history'] = data[data.variable_id].attrs.get('history','') + f', clim mean {t0}-{t1}'
            if wildcards.freq=='2010':
                import cftime
                import pandas as pd
                data = data.rename(month='time')
                cftimes = cftime.date2num(pd.date_range('2010-01-31','2010-12-31', freq='M').to_list(),
                                                  'days since 2010-01-01', 
                                                      has_year_zero=False, calendar = 'gregorian')
                data = data.assign_coords(time=cftimes)
                data.time.attrs['units'] = 'days since 2010-01-01'
            dvar_attrs = copy_meta_data_CMIP(data[wildcards.variable].attrs)

        elif freq == 'Amon':
            data = ds
        else:
            raise(ValueError(f'{wildcards.freq} is an invalid frequency'))
    return data


In [53]:
data = resample_data(ds_load)

In [54]:
data = data.compute()

In [55]:
if snakemake.config.get('regrid_params', None) and snakemake.params.get('regrid', True):
    grid_params = snakemake.config['regrid_params']
    grid_path = grid_params.get('grid_path',None)
    method=grid_params.get('method','conservative')
    if grid_path:
        out_grid = xr.open_dataset(grid_path)
        data = regrid_global(data, out_grid, method=method)
    elif grid_params.get('dxdy',None):
        dxdy = grid_params['dxdy']
        data = regrid_global(data, lon=dxdy[0], lat=dxdy[1], method=method)
    else:
        print('No outgrid provided!')

In [56]:
data.attrs = {**data.attrs,**attrs}



In [57]:
data = data.assign_attrs(variable_id='loaddust')
data.attrs['source'] = ', '.join(snakemake.input)
data.attrs['history'] = f'@{time.ctime()} Generated by: {pl.Path.cwd().parts[-1]}'
data.attrs['frequency'] = snakemake.wildcards.freq
data.to_netcdf(snakemake.output.outpath[0])